In [1]:
import subprocess
import pandas as pd

# Get file names as the nodes
Be carefull, all the file names must not contain Chinese characters.

In [34]:
PATH = r'/Users/young/Desktop/paper/collections/Feng\ Han/105paperproducts'
cmd = r'ls %s | egrep "^\d.*"' % (PATH)
catalog = [d for d in subprocess.check_output(cmd, shell=True).decode('utf-8').split('\n')][:-1]
cmd = r'find %s -type f| egrep ".*jpg"'%PATH
products105 = [d.split('/')[-1].split('.')[0] for d in subprocess.check_output(cmd, shell=True).decode('utf-8').split('\n')][:-1]


In [56]:
print('directories: ', catalog[:5])
print('files: ', products105[:5])

directories:  ['11book', '17package_container', '1bookmarkleaflet', '1calligraph', '1handcraft']
files:  ['book1', 'book10', 'book11', 'book2', 'book3']


# Define the nodes at each level

In [36]:
top_level = [
    'Preprinted'
]
physic_features = [
    'R.orientation',
    'R.size',
    'R.spatiality',
    'R.texture',
    'R.permeability'
]
information_features = [
    'reorganize',
    'cheap',
    'protable',
    'digitalized'
]
down_level = [
    'user',
    'AR',
]

# Save csv files which define all the weights of links
You can later modify the weights at file `first_second.csv`(which defines the links between first and second level), `second_third.csv`, etc.

<span style="color:red"><B>ATTENTION:</B></span> Running the follow commented codes <span style="color:red"><B>WILL OVERWRITE</B></span> your previous setting.

In [47]:
"""
this block defines the structure of dataframe files
"""
# pd.DataFrame(1, index=products105, columns=(top_level)).to_csv('../data/first_second.csv')
# pd.DataFrame(1, index=top_level, columns=(catalog)).to_csv('../data/second_third.csv')
# pd.DataFrame(1, index=catalog, columns=(physic_features)).to_csv('../data/third_fourth.csv')
# pd.DataFrame(1, index=physic_features, columns=(information_features)).to_csv('../data/fourth_fifth.csv')   
# pd.DataFrame(1, index=information_features, columns=(down_level)).to_csv('../data/fifth_sixth.csv')  

Ever time you change the `*.csv` file, you should <b> run the following code</b> so as to apply the change in the `link.json ` file which defines the final data of the sankey graph.

In [60]:
def link_generator(df):
    for i, row in df.iterrows():
        for j, cell in row.iteritems():
            if cell==1:
                yield dict(source=i, target=j, value=1)

links = pd.DataFrame()
# change this to modify different level of the sankey graph
for file in ['first_second', 'second_third', 'third_fourth', 'fourth_fifth', 'fifth_sixth']:  
    csv_file = pd.read_csv('../data/%s.csv'%file, index_col=0)
    links = pd.concat([links, pd.DataFrame(list(link_generator(csv_file)))])
links.to_json(orient='values', path_or_buf='../data/links.json')
links.head()

,source,target,value
0,book1,Preprinted,1
1,book1,Modifiable,1
2,book1,User_generated,1
3,book10,Preprinted,1
4,book10,Modifiable,1


# Other solutions for drawing the Sankey chart (deprecate in this project)

In [30]:
from ipysankeywidget import SankeyWidget
from ipywidgets import Layout
from IPython.display import (
    Image,
    SVG
)

layout = Layout(width="1000", height="800")
sankey = SankeyWidget(links=flows.to_dict('records'), layout=layout)
sankey
sankey.auto_save_png('test.png')

SankeyWidget(layout=Layout(height='800', width='1000'), links=[{'source': 'Preprinted', 'target': '11book', 'v…

In [62]:
import plotly.plotly as py
import plotly
import plotly.io as pio
# from plotly.offline import iplot, init_notebook_mode


data = dict(
    type='sankey',
    node = dict(
      pad = 15,
      thickness = 20,
      line = dict(
        color = "black",
        width = 0.5
      ),
      label = ["A1", "A2", "B1", "B2", "C1", "C2"],
      color = ["blue", "blue", "blue", "blue", "blue", "blue"]
    ),
    link = dict(
      source = [0,1,0,2,3,3],
      target = [2,3,3,4,4,5],
      value = [8,4,2,8,4,2]
  ))

layout =  dict(
    title = "Basic Sankey Diagram",
    font = dict(
      size = 10
    )
)

fig = dict(data=[data], layout=layout)
py.iplot(fig, validate=False)
# pio.write_image(fig, 'fig1.pdf')